# Appendix B

## Data Download and Merge

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from constants import *

Here, we pull data from a remote source and save it locally. This is better done with manually pulling from the mirror in file explorer or a web browser depending on which mirror you are using due to sponsorTimes.csv being ~4.7GB in size. We let python garbage collect the variables here, and load from local files later so we don't need to rerun this every time. No manipulation of the data is done here other than merging into one file and selecting correct data types. This is done on a desktop with 32GB of RAM. On our laptops with less, this tends to max out system memory and degrade to using a swap file.

In [2]:
download = False # Caution sponsorTimes.csv is 4GB
if download:
    for name in files:
        df = pd.read_csv(mirror_url+name)
        df.to_csv(data_folder+name)

From here, we will load the data from the local raw files.

In [3]:
def convert_timestamp_to_seconds(ts):
    """timeSubmitted is a unix time stamp in ms
    we convert to seconds to avoid integer overflow.
    """
    return int(ts) / 1000

dfs = {}
for name in files:
    print(name)
    df = pd.read_csv(data_folder+name, 
                     dtype=dtypes_,
                     usecols=lambda col: col not in unwanted_cols_,
                     converters={'timeSubmitted':convert_timestamp_to_seconds}
                    )
    # attempt to compress some of the columns.
    # for col in dtypes_sparse:
    #      if col in df.columns:
    #          df[col] = df[col].astype(pd.SparseDtype("string"))
    dfs[name.removesuffix('.csv')] = df
    display(df.head())

sponsorTimes.csv


,videoID,startTime,endTime,votes,locked,incorrectVotes,UUID,userID,timeSubmitted,views,category,actionType,videoDuration,hidden,reputation,shadowHidden,description
0,fBxtS9BpVWs,714.00000,763.00000,64,False,1,7c9619bd-8d91-4b42-a9be-39ffc99dad4a,38e7c2af-09f4-4492-bf49-75e443962ccd,1.564089e+09,386,sponsor,skip,0.0,0,0.0,False,<NA>
1,FfgT6zx4k3Q,446.51013,513.39233,225,False,1,96150fa0-a28a-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1.564089e+09,3222,sponsor,skip,0.0,1,0.0,False,<NA>
2,9P6rdqiybaw,488.52150,542.11035,-2,False,1,81024780-a367-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1.564089e+09,25661,sponsor,skip,552.0,0,0.0,False,<NA>
3,UjtOGPJ0URM,497.14310,569.15330,-2,False,1,b3237a80-a2c3-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1.564089e+09,24217,sponsor,skip,0.0,0,0.0,False,<NA>
4,fBxtS9BpVWs,41.00000,53.00000,115,False,1,b2465943-1313-449c-b75c-08b14756ac0a,38e7c2af-09f4-4492-bf49-75e443962ccd,1.564089e+09,768,sponsor,skip,0.0,0,0.0,False,<NA>


userNames.csv


,userID,userName,locked
0,6da67183d9b987cffe30dd44cfa53a4fb26bf22067c5c9...,RHG,False
1,950f2c3202ff0835ac683a1e97632d71257eeb5a9cc9c7...,Scuddzero,False
2,c4a6408834ac21d6bd8eca3cee787a1b1c3009ffacb1d4...,Geroyuni,False
3,01c70fbb8cebe46f5ca2b81e0c1fe8d490709d86e2624e...,Anonymous SponsorBlock Users,False
4,d45b223dd02b0c6c0019a3d7aa5675245375095fa42345...,Cool People,False


videoInfo.csv


,videoID,channelID,title,published
0,QyTRvbb3gkk,UCr7tNSNf7_aEEh5P-F5mE4A,Kronii Got Friendzoned by Chat but She End Up ...,1.634429e+09
1,qU1Yv58EXcc,UCo_IB5145EVNcf8hw1Kku7w,Game Theory: Minecraft's DARKEST Timeline! (He...,1.634429e+09
2,yKkVHBh9DQk,UCXJkLU1wZVqZjjVe1MuRj-A,TRIPLE RECORD EN GLOBILLOS? 🎈,1.633565e+09
3,7wCZSBOX7eM,UCg83RGdRpwfvoFEuE2zWKZA,Johnny vs. Nickelodeon All-Star Brawl (Sponsored),1.633392e+09
4,VVGjjaWWeRA,UCKBYXp4Xn2I2tL1UL4fpbhw,WOTB | NEW BIG HITTING JAGTIGER PREMIUM!,1.634429e+09


vipUsers.csv


,userID
0,57ddecc5b36813ddb8ea1eba73342c8a783527b884b6eb...
1,c4a6408834ac21d6bd8eca3cee787a1b1c3009ffacb1d4...
2,d261c35ce21b0554c183fc42e2f92bf30609e0540bad8e...
3,963cdc21439055c825860792fa6ef0a48ffc8823f20f31...
4,7b89ea26f77bda8176e655eee86029f28c1e6514b6d6e3...


Here, we first merge the videoInfo into the sponsorTime file, so every sponsor segment has details about the appriopriate video. The 'published' time feature will get cast to 64bit floats so that NaN can be used for segments that don't have video info available. 64-bit floats are sufficient precision for our application, so we won't cast back to int. We then add an isVIP column and merge VIP user data into sponsor and fill all other non-VIP users with False. userNames won't get merged into the main file since they have limited use, but may be used later for visualizing most active users.

In [4]:
merged = dfs['sponsorTimes'].merge(dfs['videoInfo'], on='videoID', how='left')

dfs['vipUsers']['isVIP'] = True
dfs['vipUsers']['isVIP'] = dfs['vipUsers']['isVIP'].astype('boolean')
merged = merged.merge(dfs['vipUsers'], on='userID', how='left')
merged['isVIP'] = merged['isVIP'].fillna(False)

del dfs

pd.set_option('display.max_columns', None)
merged.head()

,videoID,startTime,endTime,votes,locked,incorrectVotes,UUID,userID,timeSubmitted,views,category,actionType,videoDuration,hidden,reputation,shadowHidden,description,channelID,title,published,isVIP
0,fBxtS9BpVWs,714.00000,763.00000,64,False,1,7c9619bd-8d91-4b42-a9be-39ffc99dad4a,38e7c2af-09f4-4492-bf49-75e443962ccd,1.564089e+09,386,sponsor,skip,0.0,0,0.0,False,<NA>,UCXuqSBlHAE6Xw-yeJA0Tunw,Really Stupid & Expensive Gaming Setup!,1.529539e+09,False
1,FfgT6zx4k3Q,446.51013,513.39233,225,False,1,96150fa0-a28a-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1.564089e+09,3222,sponsor,skip,0.0,1,0.0,False,<NA>,UCsXVk37bltHxD1rDPwtNM8Q,Could Your Phone Hurt You? Electromagnetic Pol...,1.562458e+09,False
2,9P6rdqiybaw,488.52150,542.11035,-2,False,1,81024780-a367-11e9-b256-cb886cabe693,38e7c2af-09f4-4492-bf49-75e443962ccd,1.564089e+09,25661,sponsor,skip,552.0,0,0.0,False,<NA>,UCsXVk37bltHxD1rDPwtNM8Q,Wormholes Explained – Breaking Spacetime,1.534032e+09,False
3,UjtOGPJ0URM,497.14310,569.15330,-2,False,1,b3237a80-a2c3-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1.564089e+09,24217,sponsor,skip,0.0,0,0.0,False,<NA>,UCsXVk37bltHxD1rDPwtNM8Q,Why Alien Life Would be our Doom - The Great F...,1.517443e+09,False
4,fBxtS9BpVWs,41.00000,53.00000,115,False,1,b2465943-1313-449c-b75c-08b14756ac0a,38e7c2af-09f4-4492-bf49-75e443962ccd,1.564089e+09,768,sponsor,skip,0.0,0,0.0,False,<NA>,UCXuqSBlHAE6Xw-yeJA0Tunw,Really Stupid & Expensive Gaming Setup!,1.529539e+09,False


In [5]:
merged.to_csv(sb_data_file)